In [1]:
import configparser

import pandas as pd
from binance import Client

CFG_FL_NAME = "../user.cfg"
USER_CFG_SECTION = "binance_user_config"
config = configparser.ConfigParser()
config.read(CFG_FL_NAME)

api_key = config.get(USER_CFG_SECTION, "api_key")
api_secret = config.get(USER_CFG_SECTION, "api_secret_key")
client = Client(api_key, api_secret)

print("Client created", client)

Client created <binance.client.Client object at 0x00000253E9E693A0>


In [2]:
from datetime import datetime, timedelta


# download ohlc candle at specific timestamp from binance
def download_ohlc_1_min_candle_from_binance_for_pair_at_timestamp(pair:str,timestamp:int):

    # download the klines (OHLC candles) for the given symbol and timestamp
    klines = client.get_historical_klines(
        symbol=pair,
        interval=Client.KLINE_INTERVAL_1MINUTE,
        start_str=timestamp,
        end_str= timestamp + 1*60*1000# 1 minute, 1 candle
    )
    # extract the closing price from the klines
    closing_price = float(klines[0][4])

    return closing_price

## test above function
sample_time = "22:57:51 09/11/2022" # 2022-11-09 15:57:51.295978
local_time = datetime.strptime(sample_time, "%H:%M:%S %d/%m/%Y")
# Convert the local time to GMT time
gmt_time = local_time - timedelta(hours=7)

print(f'local : {local_time}')
print(f'gmt : {gmt_time}')

sample_timestamp = int(gmt_time.timestamp()) * 1000

a_coin = 'MANA'
b_coin = 'SOL'

a = download_ohlc_1_min_candle_from_binance_for_pair_at_timestamp(f'{a_coin}USDT', sample_timestamp)
b = download_ohlc_1_min_candle_from_binance_for_pair_at_timestamp(f'{b_coin}USDT', sample_timestamp)

ratio = a/b

print(f'{a_coin} = {a} , {b_coin} = {b}, ratio : {a_coin}/{b_coin} = {ratio}')

local : 2022-11-09 22:57:51
gmt : 2022-11-09 15:57:51
MANA = 0.5437 , SOL = 18.55, ratio : MANA/SOL = 0.0293099730458221


In [3]:
import sys

sys.path.append('../')

import os
import sqlite3

ROOT_PATH = r'C:\AgodaGit\binance-trade-bot'
db_file_path = os.path.join(ROOT_PATH, "data2/crypto_trading.db")
con = sqlite3.connect(db_file_path)
cur = con.cursor()

In [4]:
enabled_coins = cur.execute('SELECT symbol from coins where enabled = 1').fetchall()

enabled_coins =  [item for tpl in enabled_coins for item in tpl]
enabled_coins

['ETH',
 'ADA',
 'XRP',
 'EOS',
 'XLM',
 'ETC',
 'VET',
 'LINK',
 'THETA',
 'MATIC',
 'DOGE',
 'SOL',
 'MANA',
 'SAND',
 'DOT',
 'RUNE',
 'FIL']

In [7]:
from datetime import datetime
from pandas import json_normalize


def convert_to_unix_timestamp(date_time_str):
    date_obj = datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')
    timestamp = int(date_obj.timestamp()) * 1000
    return timestamp


symbol = 'SOLUSDT'

start_from_timestamp = convert_to_unix_timestamp('2022-11-09 15:57:51.295')


def download_spot_trading_history_for_pair(pair: str, frm: int):
    trades = client.get_my_trades(symbol=pair, startTime=frm)
    trades_df = json_normalize(trades)
    df = trades_df#.query(f'isBuyer == {is_buyer}')
    df['timestamp'] = pd.to_datetime(df['time'], unit='ms')
    df[['price', 'qty', 'quoteQty']] = df[['price', 'qty', 'quoteQty']].apply(
        lambda x: pd.to_numeric(x, errors='coerce'))
    df.sort_values(by='time', ascending=False, inplace=True)
    df['buy_sell'] = df['isBuyer'].apply(lambda x: 'BUY' if x else 'SELL')
    return df
    # return pd.DataFrame(df, columns=['price', 'buy_sell', 'qty', 'quoteQty', 'time', 'timestamp'])


t = download_spot_trading_history_for_pair(symbol, start_from_timestamp)

t

,symbol,id,orderId,orderListId,price,qty,quoteQty,commission,commissionAsset,time,isBuyer,isMaker,isBestMatch,timestamp,buy_sell
2,SOLUSDT,334827949,3487139398,-1,13.82,1.36,18.7952,0.00005739,BNB,1672762145453,False,False,True,2023-01-03 16:09:05.453,SELL
0,SOLUSDT,323296673,3382922668,-1,15.87,0.25,3.9675,0.00001051,BNB,1668009471177,True,False,True,2022-11-09 15:57:51.177,BUY
1,SOLUSDT,323296674,3382922668,-1,15.87,0.53,8.4111,0.00002229,BNB,1668009471177,True,False,True,2022-11-09 15:57:51.177,BUY


In [8]:
plan = ['SOL','MANA','SAND']

result = download_spot_trading_history_for_pair(f'{plan[0]}USDT',start_from_timestamp)

for p in plan[1:]:
    result = result.append(download_spot_trading_history_for_pair(f'{p}USDT',start_from_timestamp))

result.sort_values(by='time', ascending=False, inplace=True)
result

C:\Users\skhan\AppData\Local\Temp\ipykernel_3488\1748545717.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(download_spot_trading_history_for_pair(f'{p}USDT',start_from_timestamp))
C:\Users\skhan\AppData\Local\Temp\ipykernel_3488\1748545717.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(download_spot_trading_history_for_pair(f'{p}USDT',start_from_timestamp))


,symbol,id,orderId,orderListId,price,qty,quoteQty,commission,commissionAsset,time,isBuyer,isMaker,isBestMatch,timestamp,buy_sell
0,SANDUSDT,175480805,2387167116,-1,0.6232,49.00,30.5368,0.00007665,BNB,1673688040670,True,False,True,2023-01-14 09:20:40.670,BUY
2,MANAUSDT,179975016,1833425514,-1,0.5601,55.00,30.8055,0.00007688,BNB,1673688040576,False,False,True,2023-01-14 09:20:40.576,SELL
1,MANAUSDT,179111309,1821312376,-1,0.3091,60.00,18.5460,0.00005633,BNB,1672762146638,True,False,True,2023-01-03 16:09:06.638,BUY
2,SOLUSDT,334827949,3487139398,-1,13.8200,1.36,18.7952,0.00005739,BNB,1672762145453,False,False,True,2023-01-03 16:09:05.453,SELL
0,SOLUSDT,323296673,3382922668,-1,15.8700,0.25,3.9675,0.00001051,BNB,1668009471177,True,False,True,2022-11-09 15:57:51.177,BUY
1,SOLUSDT,323296674,3382922668,-1,15.8700,0.53,8.4111,0.00002229,BNB,1668009471177,True,False,True,2022-11-09 15:57:51.177,BUY
0,MANAUSDT,177291539,1784290174,-1,0.5160,24.00,12.3840,0.00003193,BNB,1668009471083,False,False,True,2022-11-09 15:57:51.083,SELL


In [9]:
tmp = pd.DataFrame(result, columns=['symbol', 'buy_sell', 'timestamp'])
tmp

,symbol,buy_sell,timestamp
0,SANDUSDT,BUY,2023-01-14 09:20:40.670
2,MANAUSDT,SELL,2023-01-14 09:20:40.576
1,MANAUSDT,BUY,2023-01-03 16:09:06.638
2,SOLUSDT,SELL,2023-01-03 16:09:05.453
0,SOLUSDT,BUY,2022-11-09 15:57:51.177
1,SOLUSDT,BUY,2022-11-09 15:57:51.177
0,MANAUSDT,SELL,2022-11-09 15:57:51.083


In [10]:
class Trade:
    def __init__(self, alt_coin_id, crypto_coin_id, selling, state, alt_starting_balance, alt_trade_amount, crypto_starting_balance, crypto_trade_amount, datetime):
        self.alt_coin_id = alt_coin_id
        self.crypto_coin_id = crypto_coin_id
        self.selling = selling
        self.state = state
        self.alt_starting_balance = alt_starting_balance
        self.alt_trade_amount = alt_trade_amount
        self.crypto_starting_balance = crypto_starting_balance
        self.crypto_trade_amount = crypto_trade_amount
        self.datetime = datetime

    # def __str__(self):
    #     return f"Trade(alt_coin_id={self.alt_coin_id}, crypto_coin_id={self.crypto_coin_id}, selling={self.selling}, state={self.state}, alt_starting_balance={self.alt_starting_balance}, alt_trade_amount={self.alt_trade_amount}, crypto_starting_balance={self.crypto_starting_balance}, crypto_trade_amount={self.crypto_trade_amount}, datetime={self.datetime})"

    def __str__(self):
        return f"{self.alt_coin_id}, {self.selling}, {self.alt_trade_amount}, {self.crypto_trade_amount}, datetime={self.datetime}"

    def to_tuple(self):
        return (self.alt_coin_id, self.crypto_coin_id, self.selling, self.state, self.alt_starting_balance, self.alt_trade_amount, self.crypto_starting_balance, self.crypto_trade_amount, self.datetime)


In [18]:
result.sort_values(by='time', ascending=True, inplace=True)

last_symbol = ''

rows = []

current_coin = ''
for index, row in result.iterrows():
    symbol = row['symbol'].replace('USDT','')

    if current_coin == '' or current_coin != symbol:
        current_trade_history = Trade(
            alt_coin_id=symbol,
            crypto_coin_id='USDT',
            selling = not row['isBuyer'],
            state='COMPLETE',
            alt_starting_balance=row['qty'],
            alt_trade_amount=row['qty'],
            crypto_starting_balance=row['quoteQty'],
            crypto_trade_amount=row['quoteQty'],
            datetime=row['timestamp']
        )
        rows.append(current_trade_history)
        current_coin = symbol
    else:
        current_trade_history:Trade = rows[-1]
        current_trade_history.alt_starting_balance += row['qty']
        current_trade_history.alt_trade_amount += row['qty']

        current_trade_history.crypto_starting_balance += row['quoteQty']
        current_trade_history.crypto_trade_amount += row['quoteQty']

    #print(current_trade_history)

for r in rows:
    print(r.to_tuple())

('MANA', 'USDT', True, 'COMPLETE', 24.0, 24.0, 12.384, 12.384, Timestamp('2022-11-09 15:57:51.083000'))
('SOL', 'USDT', False, 'COMPLETE', 2.14, 2.14, 31.1738, 31.1738, Timestamp('2022-11-09 15:57:51.177000'))
('MANA', 'USDT', False, 'COMPLETE', 115.0, 115.0, 49.3515, 49.3515, Timestamp('2023-01-03 16:09:06.638000'))
('SAND', 'USDT', False, 'COMPLETE', 49.0, 49.0, 30.5368, 30.5368, Timestamp('2023-01-14 09:20:40.670000'))


In [20]:
cur.execute('SELECT * FROM trade_history ORDER BY datetime DESC LIMIT 1')
last_row = cur.fetchone()

last_index = last_row[0]

cur.close()
con.close()

ProgrammingError: Cannot operate on a closed cursor.

In [21]:
t = rows[0].datetime
int(t.timestamp())

1668009471

In [22]:
conn = sqlite3.connect(db_file_path)
cursor = conn.cursor()
insert_list = [(last_index + idx + 1,
                coin.alt_coin_id,
                coin.crypto_coin_id,
                coin.selling,
                coin.state,
                coin.alt_starting_balance,
                coin.alt_trade_amount,
                coin.crypto_starting_balance,
                coin.crypto_trade_amount,
                coin.datetime.strftime('%Y-%m-%d %H:%M:%S')
                ) for idx, coin in enumerate(rows)]

insert_command = "INSERT INTO trade_history VALUES (?,?,?,?,?,?,?,?,?,?)"
for i in insert_list:
    #print(insert_command)
    cursor.execute(insert_command,i)

conn.commit()
conn.close()


In [23]:
coin_price = {}
time =  "2023-01-14 09:20:40" ## this is when We LAST jumped
timestamp = int(datetime.strptime(time, "%Y-%m-%d %H:%M:%S").timestamp()) * 1000
for coin in enabled_coins:
    coin_price[coin] = download_ohlc_1_min_candle_from_binance_for_pair_at_timestamp(f'{coin}USDT',timestamp)


# calculate all ratios
pairs = {}
for from_coin in enabled_coins:
    for to_coin in enabled_coins:
        if from_coin != to_coin:
            pair = (from_coin,to_coin)
            ratio = coin_price[from_coin] / coin_price[to_coin]
            pairs[pair] = ratio

pairs

{('ETH', 'ADA'): 4335.484772282761,
 ('ETH', 'XRP'): 3858.9156926137775,
 ('ETH', 'EOS'): 1466.6068052930057,
 ('ETH', 'XLM'): 17814.810562571758,
 ('ETH', 'ETC'): 66.65249140893471,
 ('ETH', 'VET'): 74707.27010110737,
 ('ETH', 'LINK'): 225.86171761280931,
 ('ETH', 'THETA'): 1688.4330794341677,
 ('ETH', 'MATIC'): 1583.3367346938776,
 ('ETH', 'DOGE'): 17582.662889518415,
 ('ETH', 'SOL'): 69.86357496623143,
 ('ETH', 'MANA'): 2759.5056019918193,
 ('ETH', 'SAND'): 2389.022324865281,
 ('ETH', 'DOT'): 274.6318584070796,
 ('ETH', 'RUNE'): 891.764367816092,
 ('ETH', 'FIL'): 355.6429062571625,
 ('ADA', 'ETH'): 0.00023065471395335347,
 ('ADA', 'XRP'): 0.8900770952499378,
 ('ADA', 'EOS'): 0.3382797731568998,
 ('ADA', 'XLM'): 4.109070034443169,
 ('ADA', 'ETC'): 0.015373711340206184,
 ('ADA', 'VET'): 17.231584015406835,
 ('ADA', 'LINK'): 0.052096069868995634,
 ('ADA', 'THETA'): 0.38944504896626764,
 ('ADA', 'MATIC'): 0.36520408163265305,
 ('ADA', 'DOGE'): 4.055524079320113,
 ('ADA', 'SOL'): 0.01611

In [5]:
import sqlite3

# Connect to source database and retrieve cursor
src_conn = sqlite3.connect(r"C:\AgodaGit\binance-trade-bot\data2\crypto_trading.db")
src_cursor = src_conn.cursor()

# Connect to target database and retrieve cursor
tgt_conn = sqlite3.connect(r"C:\AgodaGit\binance-trade-bot\data\crypto_trading.db")
tgt_cursor = tgt_conn.cursor()

# Define table names to copy
table_names = ["coins", "current_coin_history", "pairs", "trade_history"]

# Loop through each table and copy data
for table_name in table_names:
    # Delete existing data in target table
    tgt_cursor.execute(f"DELETE FROM {table_name}")

    # Copy data from source table to target table
    print(f'Copying from {table_name}')
    src_cursor.execute(f"SELECT * FROM {table_name}")
    rows = src_cursor.fetchall()
    tgt_cursor.executemany(f"INSERT INTO {table_name} VALUES ({','.join('?' * len(rows[0]))})", rows)

# Commit changes and close connections
tgt_conn.commit()
tgt_conn.close()
src_conn.close()

Copying from coins
Copying from current_coin_history
Copying from pairs
Copying from trade_history
